# Tutorial 6: Liquidity and AMMs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/blockchain-tokens-and-incentives/blob/main/notebooks/tutorial_06_liquidity_and_amms.ipynb)

---

> *"The exchange does not sleep. It does not favor. It does not forget. It only computes: x times y equals k. Buy or sell, the curve decides your price."*
>
> — Brenn Auster, *On Automated Exchange* (Year 871)

---

## The Problem of Prices

Tutorials 01-05 solved the trust problem: staking makes honesty dominant, tokens represent value, slashing punishes misbehavior. But there's a deeper problem Auster hadn't addressed: **how do you set prices?**

The guild's commodity markets relied on human market-makers — merchants who held inventory of both sides of a trade and set bid/ask prices based on experience, rumor, and instinct. Near the Capital, this worked. Near the Dens, it didn't. Prices shifted without explanation. A merchant who set iron_ore at 2.5 salt_crystal yesterday might find the ratio was 4.1 today — not because supply changed, but because the Dens distorted valuation itself.

Auster needed an exchange mechanism that:
1. **Always has liquidity** — never runs out of either commodity
2. **Sets prices automatically** — based on supply and demand, not human judgment
3. **Requires no trusted market-maker** — anyone can provide liquidity
4. **Self-corrects** — prices adjust continuously as trades occur

The answer was the **Automated Market Maker (AMM)** — specifically, the **constant product** formula used by Uniswap. Instead of matching buyers and sellers, the AMM holds reserves of both commodities and computes prices mathematically. The formula is astonishingly simple: **x × y = k**.

## Learning Objectives

In this tutorial, you will:

1. **Understand the constant product formula** — x × y = k and why it works
2. **Build a Uniswap-style AMM** — swaps, liquidity provision, LP tokens
3. **Analyze the AMM dataset** — 200 pool state transitions across 4 commodity pairs
4. **Compute impermanent loss** — the hidden cost of providing liquidity
5. **Model slippage** — why large trades get worse prices than small ones

**Technical Concepts:**
- Constant product market makers (x × y = k)
- Liquidity pools and LP tokens
- Swap mechanics and fee structure
- Impermanent loss and liquidity provider economics
- Price impact and slippage

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

amm_pools = pd.read_csv(BASE_URL + "amm_liquidity_pools.csv")
guild_trades = pd.read_csv(BASE_URL + "guild_trade_ledger.csv")

print(f"AMM pool states: {len(amm_pools)} transitions")
print(f"Pools: {amm_pools['pool'].nunique()}")
print(f"Actions: {list(amm_pools['action'].unique())}")
print(f"\nPool pairs:")
for pool in amm_pools['pool'].unique():
    count = len(amm_pools[amm_pools['pool'] == pool])
    print(f"  {pool:35s} {count} states")

## The Constant Product Formula

An AMM holds **reserves** of two tokens (commodities). The price relationship between them is governed by a single equation:

$$x \times y = k$$

Where:
- **x** = reserve of token A (e.g., iron_ore)
- **y** = reserve of token B (e.g., salt_crystal)
- **k** = constant product (invariant)

When someone buys token A, they deposit token B. This increases y and must decrease x to keep k constant. The price emerges from this constraint — it's the ratio of reserves.

**Spot price of A in terms of B:** `price_A = y / x`

The beauty: no order book, no market-maker, no price oracle. Just arithmetic.

In [ ]:
# Visualize the constant product curve
k = 10000  # Example constant product

x = np.linspace(10, 500, 1000)
y = k / x

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# The curve
ax1.plot(x, y, 'b-', linewidth=2)
ax1.set_xlabel('Reserve A (iron_ore)')
ax1.set_ylabel('Reserve B (salt_crystal)')
ax1.set_title(f'Constant Product Curve (k = {k:,})')

# Mark a swap: buying 20 units of A
x0, y0 = 100, k / 100  # Starting point
x1 = x0 - 20  # After removing 20 units of A
y1 = k / x1    # New y to maintain k
dy = y1 - y0   # Cost in B

ax1.plot(x0, y0, 'go', markersize=10, label=f'Before: ({x0}, {y0:.0f})')
ax1.plot(x1, y1, 'ro', markersize=10, label=f'After: ({x1}, {y1:.0f})')
ax1.annotate('', xy=(x1, y1), xytext=(x0, y0),
             arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax1.legend()
ax1.grid(alpha=0.3)

# Price impact: how much does the price change per unit bought?
amounts = np.arange(1, 60)
prices = []
for amt in amounts:
    new_x = x0 - amt
    new_y = k / new_x
    cost = new_y - y0
    avg_price = cost / amt  # Average price per unit
    prices.append(avg_price)

spot_price = y0 / x0  # Marginal price at current reserves
ax2.plot(amounts, prices, 'b-', linewidth=2, label='Average execution price')
ax2.axhline(y=spot_price, color='green', linestyle='--', alpha=0.7, label=f'Spot price: {spot_price:.2f}')
ax2.set_xlabel('Amount of A purchased')
ax2.set_ylabel('Average price (B per A)')
ax2.set_title('Price Impact: Larger Trades Get Worse Prices')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Swap example: buy 20 iron_ore")
print(f"  Starting reserves: {x0} iron, {y0:.0f} salt")
print(f"  Ending reserves:   {x1} iron, {y1:.0f} salt")
print(f"  Cost: {dy:.1f} salt_crystal for 20 iron_ore")
print(f"  Spot price: {spot_price:.2f} salt per iron")
print(f"  Execution price: {dy/20:.2f} salt per iron (worse due to price impact)")

## Building a Uniswap-Style AMM

Let's implement a complete AMM with:
- **Swaps** — trade one commodity for another
- **Liquidity provision** — deposit both commodities to earn fees
- **LP tokens** — receipt tokens representing pool shares
- **Fee collection** — 0.3% per swap, distributed to LPs

In [ ]:
class ConstantProductAMM:
    """Uniswap V2-style constant product AMM."""
    
    FEE_RATE = 0.003  # 0.3% swap fee
    
    def __init__(self, token_a, token_b):
        self.token_a = token_a
        self.token_b = token_b
        self.reserve_a = 0.0
        self.reserve_b = 0.0
        self.total_lp_tokens = 0.0
        self.lp_balances = defaultdict(float)  # provider → LP tokens
        self.total_fees_a = 0.0
        self.total_fees_b = 0.0
        self.history = []
    
    @property
    def k(self):
        return self.reserve_a * self.reserve_b
    
    @property
    def price_a_in_b(self):
        if self.reserve_a == 0:
            return 0
        return self.reserve_b / self.reserve_a
    
    @property
    def price_b_in_a(self):
        if self.reserve_b == 0:
            return 0
        return self.reserve_a / self.reserve_b
    
    def add_liquidity(self, provider, amount_a, amount_b):
        """Add liquidity. First deposit sets the ratio; subsequent must match."""
        if self.total_lp_tokens == 0:
            # First deposit: LP tokens = sqrt(amount_a * amount_b)
            lp_minted = (amount_a * amount_b) ** 0.5
        else:
            # Proportional deposit
            ratio_a = amount_a / self.reserve_a
            ratio_b = amount_b / self.reserve_b
            lp_minted = min(ratio_a, ratio_b) * self.total_lp_tokens
        
        self.reserve_a += amount_a
        self.reserve_b += amount_b
        self.total_lp_tokens += lp_minted
        self.lp_balances[provider] += lp_minted
        
        self._log('add_liquidity', provider, amount_a, amount_b, lp_minted)
        return lp_minted
    
    def remove_liquidity(self, provider, lp_amount):
        """Burn LP tokens and withdraw proportional reserves."""
        assert self.lp_balances[provider] >= lp_amount, "Insufficient LP tokens"
        
        share = lp_amount / self.total_lp_tokens
        amount_a = self.reserve_a * share
        amount_b = self.reserve_b * share
        
        self.reserve_a -= amount_a
        self.reserve_b -= amount_b
        self.total_lp_tokens -= lp_amount
        self.lp_balances[provider] -= lp_amount
        
        self._log('remove_liquidity', provider, amount_a, amount_b, -lp_amount)
        return amount_a, amount_b
    
    def swap_a_for_b(self, trader, amount_a_in):
        """Swap token A for token B. Returns amount of B received."""
        fee = amount_a_in * self.FEE_RATE
        amount_a_after_fee = amount_a_in - fee
        self.total_fees_a += fee
        
        # Constant product: (reserve_a + da) * (reserve_b - db) = k
        # db = reserve_b - k / (reserve_a + da)
        new_reserve_a = self.reserve_a + amount_a_after_fee
        new_reserve_b = self.k / new_reserve_a
        amount_b_out = self.reserve_b - new_reserve_b
        
        # Slippage: difference between spot price and execution price
        spot_price = self.price_a_in_b
        exec_price = amount_b_out / amount_a_in
        slippage = (spot_price - exec_price) / spot_price * 100
        
        self.reserve_a = new_reserve_a + fee  # Fee stays in pool
        self.reserve_b = new_reserve_b
        
        self._log('swap', trader, amount_a_in, amount_b_out, 0, fee, slippage)
        return amount_b_out, slippage
    
    def swap_b_for_a(self, trader, amount_b_in):
        """Swap token B for token A. Returns amount of A received."""
        fee = amount_b_in * self.FEE_RATE
        amount_b_after_fee = amount_b_in - fee
        self.total_fees_b += fee
        
        new_reserve_b = self.reserve_b + amount_b_after_fee
        new_reserve_a = self.k / new_reserve_b
        amount_a_out = self.reserve_a - new_reserve_a
        
        spot_price = self.price_b_in_a
        exec_price = amount_a_out / amount_b_in
        slippage = (spot_price - exec_price) / spot_price * 100
        
        self.reserve_b = new_reserve_b + fee
        self.reserve_a = new_reserve_a
        
        self._log('swap', trader, amount_b_in, amount_a_out, 0, fee, slippage)
        return amount_a_out, slippage
    
    def _log(self, action, trader, amount_in, amount_out, lp_delta, fee=0, slippage=0):
        self.history.append({
            'action': action, 'trader': trader,
            'amount_in': round(amount_in, 4), 'amount_out': round(amount_out, 4),
            'fee': round(fee, 4), 'slippage_pct': round(slippage, 2),
            'reserve_a': round(self.reserve_a, 4), 'reserve_b': round(self.reserve_b, 4),
            'k': round(self.k, 2), 'price_a_in_b': round(self.price_a_in_b, 4),
            'total_lp': round(self.total_lp_tokens, 4),
            'tvl': round(self.reserve_a + self.reserve_b, 2)
        })
    
    def get_history(self):
        return pd.DataFrame(self.history)

print("AMM deployed.")
print(f"  Fee rate: {ConstantProductAMM.FEE_RATE:.1%}")
print(f"  Formula: x * y = k (constant product)")
print(f"  Price: spot_price_A = reserve_B / reserve_A")

## Simulating a Guild Commodity Pool

Let's create an iron_ore/salt_crystal pool and simulate 30 rounds of trading.

In [ ]:
np.random.seed(871)  # Year of Auster's exchange paper

pool = ConstantProductAMM('iron_ore', 'salt_crystal')

# Initial liquidity from 3 guild members
providers = [
    ('Brenn Auster', 500, 800),
    ('Torren Gael', 300, 480),
    ('Dessa Morin', 200, 320),
]

print("=== INITIAL LIQUIDITY ===")
for name, a, b in providers:
    lp = pool.add_liquidity(name, a, b)
    print(f"  {name:15s} deposits {a} iron + {b} salt → {lp:.2f} LP tokens")

print(f"\nPool state:")
print(f"  Reserves: {pool.reserve_a:.0f} iron, {pool.reserve_b:.0f} salt")
print(f"  k = {pool.k:,.0f}")
print(f"  Price: 1 iron = {pool.price_a_in_b:.4f} salt")
print(f"  Total LP tokens: {pool.total_lp_tokens:.2f}")
print(f"  TVL: {pool.reserve_a + pool.reserve_b:.0f}")

In [ ]:
# Simulate 30 trades
traders = ['Mollen Vek', 'Vagabu Olt', 'Pemlik Tross', 'Serath Kyne',
           'Hask Berrol', 'Boffa Trent', 'Yasho Krent', 'Grigsu Haldo']

print("=== TRADING SIMULATION (30 rounds) ===")
print(f"{'#':>3s} {'Trader':15s} {'Direction':12s} {'In':>8s} {'Out':>8s} {'Slip%':>6s} {'Price':>8s}")

for i in range(30):
    trader = np.random.choice(traders)
    direction = np.random.choice(['buy_iron', 'buy_salt'], p=[0.55, 0.45])
    
    if direction == 'buy_iron':
        amount = np.random.uniform(5, 50)
        out, slip = pool.swap_b_for_a(trader, amount)
        print(f"{i+1:3d} {trader:15s} {'salt→iron':12s} {amount:8.1f} {out:8.1f} {slip:5.1f}% {pool.price_a_in_b:8.4f}")
    else:
        amount = np.random.uniform(5, 40)
        out, slip = pool.swap_a_for_b(trader, amount)
        print(f"{i+1:3d} {trader:15s} {'iron→salt':12s} {amount:8.1f} {out:8.1f} {slip:5.1f}% {pool.price_a_in_b:8.4f}")

print(f"\nFinal state:")
print(f"  Reserves: {pool.reserve_a:.1f} iron, {pool.reserve_b:.1f} salt")
print(f"  k = {pool.k:,.0f}")
print(f"  Price: 1 iron = {pool.price_a_in_b:.4f} salt")
print(f"  Fees collected: {pool.total_fees_a:.2f} iron, {pool.total_fees_b:.2f} salt")

In [ ]:
# Visualize price evolution and reserves
sim_history = pool.get_history()
swaps = sim_history[sim_history['action'] == 'swap'].reset_index(drop=True)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Price over time
axes[0, 0].plot(swaps.index, swaps['price_a_in_b'], 'b-', linewidth=2)
axes[0, 0].set_xlabel('Trade #')
axes[0, 0].set_ylabel('Price (salt per iron)')
axes[0, 0].set_title('Price Evolution')
axes[0, 0].grid(alpha=0.3)

# Reserves
axes[0, 1].plot(swaps.index, swaps['reserve_a'], 'r-', label='Iron reserve', linewidth=2)
axes[0, 1].plot(swaps.index, swaps['reserve_b'], 'b-', label='Salt reserve', linewidth=2)
axes[0, 1].set_xlabel('Trade #')
axes[0, 1].set_ylabel('Reserve amount')
axes[0, 1].set_title('Reserve Balances')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Slippage vs trade size
axes[1, 0].scatter(swaps['amount_in'], swaps['slippage_pct'], 
                    c='steelblue', alpha=0.7, edgecolors='black')
axes[1, 0].set_xlabel('Trade size')
axes[1, 0].set_ylabel('Slippage %')
axes[1, 0].set_title('Slippage vs Trade Size')
axes[1, 0].grid(alpha=0.3)

# Constant product (should stay roughly constant, growing slightly from fees)
axes[1, 1].plot(swaps.index, swaps['k'], 'g-', linewidth=2)
axes[1, 1].set_xlabel('Trade #')
axes[1, 1].set_ylabel('k (constant product)')
axes[1, 1].set_title('Constant Product k (grows from fees)')
axes[1, 1].grid(alpha=0.3)

plt.suptitle('AMM Simulation: iron_ore / salt_crystal Pool', fontsize=13)
plt.tight_layout()
plt.show()

## Analyzing the AMM Dataset

Now let's analyze the pre-generated `amm_liquidity_pools.csv` — 200 state transitions across 4 guild commodity pools.

In [ ]:
# Overview
print("AMM Dataset Overview:")
print(f"  Total transitions: {len(amm_pools)}")
print(f"  Pools: {amm_pools['pool'].nunique()}")

print(f"\nAction distribution:")
action_counts = amm_pools['action'].value_counts()
for action, count in action_counts.items():
    print(f"  {action:20s} {count:4d} ({count/len(amm_pools):.0%})")

print(f"\nPool statistics:")
pool_stats = amm_pools.groupby('pool').agg(
    transitions=('state_index', 'count'),
    avg_tvl=('pool_tvl', 'mean'),
    swaps=('action', lambda x: (x == 'swap').sum()),
    avg_slippage=('slippage_pct', lambda x: x[x > 0].mean() if (x > 0).any() else 0)
)
for pool, row in pool_stats.iterrows():
    print(f"  {pool:35s} {int(row['transitions'])} states, "
          f"{int(row['swaps'])} swaps, avg TVL={row['avg_tvl']:.0f}, "
          f"avg slip={row['avg_slippage']:.1f}%")

In [ ]:
# Price evolution across all 4 pools
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, pool_name in zip(axes.flat, amm_pools['pool'].unique()):
    pool_data = amm_pools[amm_pools['pool'] == pool_name]
    tokens = pool_name.split('/')
    
    ax.plot(pool_data['state_index'], pool_data['price_a_in_b'], 'b-', linewidth=1.5)
    swap_data = pool_data[pool_data['action'] == 'swap']
    ax.plot(swap_data['state_index'], swap_data['price_a_in_b'], 'ro', markersize=3, alpha=0.5)
    
    ax.set_title(pool_name, fontsize=10)
    ax.set_xlabel('State index')
    ax.set_ylabel(f'{tokens[1]} per {tokens[0]}')
    ax.grid(alpha=0.3)

plt.suptitle('Price Evolution Across Guild Commodity Pools', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# Constant product invariant check
# k should stay constant within each pool (except growing slightly from fees)
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

for ax, pool_name in zip(axes.flat, amm_pools['pool'].unique()):
    pool_data = amm_pools[amm_pools['pool'] == pool_name]
    k_values = pool_data['constant_product_k']
    
    ax.plot(pool_data['state_index'], k_values, 'g-', linewidth=1.5)
    ax.set_title(pool_name, fontsize=10)
    ax.set_xlabel('State index')
    ax.set_ylabel('k = x * y')
    ax.grid(alpha=0.3)
    
    # Annotate changes
    k_start = k_values.iloc[0]
    k_end = k_values.iloc[-1]
    pct_change = (k_end - k_start) / k_start * 100
    ax.annotate(f'Δk = {pct_change:+.1f}%', xy=(0.7, 0.1), xycoords='axes fraction',
                fontsize=9, color='darkgreen')

plt.suptitle('Constant Product k Over Time (changes from liquidity adds/removes)', fontsize=13)
plt.tight_layout()
plt.show()

print("k changes when liquidity is added or removed.")
print("Swaps preserve k (except for fee accumulation). Liquidity events change k directly.")

## Slippage Analysis

**Slippage** is the difference between the expected price (spot) and the actual execution price. Larger trades move the price more, so they experience more slippage. This is the AMM's natural defense against whales draining the pool.

In [ ]:
# Slippage analysis from the dataset
swaps = amm_pools[amm_pools['action'] == 'swap'].copy()
swaps['trade_size_pct'] = swaps['amount_in'] / swaps['pool_tvl'] * 100

print(f"Swap statistics:")
print(f"  Total swaps: {len(swaps)}")
print(f"  Avg slippage: {swaps['slippage_pct'].mean():.2f}%")
print(f"  Max slippage: {swaps['slippage_pct'].max():.2f}%")
print(f"  Avg fee: {swaps['fee'].mean():.2f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Slippage vs trade size (% of pool)
colors = [{'iron_ore/salt_crystal': 'steelblue', 'manuscript_vellum/amber_resin': 'darkorange',
           'copper_ingot/timber_cedar': 'forestgreen', 'dye_indigo/grain_winter': 'firebrick'
          }.get(p, 'gray') for p in swaps['pool']]

ax1.scatter(swaps['trade_size_pct'], swaps['slippage_pct'], c=colors, alpha=0.6, edgecolors='black', s=40)
ax1.set_xlabel('Trade Size (% of Pool TVL)')
ax1.set_ylabel('Slippage %')
ax1.set_title('Slippage vs Relative Trade Size')
ax1.grid(alpha=0.3)

# Slippage distribution
ax2.hist(swaps['slippage_pct'], bins=20, color='steelblue', alpha=0.8, edgecolor='black')
ax2.set_xlabel('Slippage %')
ax2.set_ylabel('Frequency')
ax2.set_title('Slippage Distribution Across All Swaps')
ax2.axvline(x=swaps['slippage_pct'].mean(), color='firebrick', linestyle='--',
            label=f'Mean: {swaps["slippage_pct"].mean():.2f}%')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Theoretical slippage model vs actual
# For a constant product AMM, slippage ≈ trade_size / (2 * reserve)
# Let's verify against the dataset

def theoretical_slippage(amount_in, reserve_in):
    """Approximate slippage for constant product AMM."""
    return amount_in / (reserve_in + amount_in) * 100

swaps['theoretical_slip'] = swaps.apply(
    lambda r: theoretical_slippage(r['amount_in'], r['reserve_a']), axis=1
)

plt.figure(figsize=(8, 8))
plt.scatter(swaps['theoretical_slip'], swaps['slippage_pct'], 
            c='steelblue', alpha=0.5, edgecolors='black')
max_val = max(swaps['theoretical_slip'].max(), swaps['slippage_pct'].max())
plt.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Perfect prediction')
plt.xlabel('Theoretical Slippage %')
plt.ylabel('Actual Slippage %')
plt.title('Theoretical vs Actual Slippage')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

correlation = swaps['theoretical_slip'].corr(swaps['slippage_pct'])
print(f"Correlation between theoretical and actual slippage: {correlation:.3f}")
print(f"The approximation works well for small trades but diverges for large ones.")

## Impermanent Loss

> *"The liquidity provider is not a merchant. He is a market itself — and markets can be bled by those who read the direction of prices before the pool adjusts."*
>
> — Brenn Auster

**Impermanent loss** is the hidden cost of providing liquidity to an AMM. When the price of the pooled assets changes relative to when you deposited, you end up with less value than if you'd simply held the assets.

The math: if the price of token A relative to token B changes by a factor of `r`, the impermanent loss is:

$$IL = \frac{2\sqrt{r}}{1 + r} - 1$$

This is always negative (a loss) for any price change in either direction.

In [ ]:
def impermanent_loss(price_ratio):
    """Calculate impermanent loss for a given price ratio change.
    price_ratio = new_price / initial_price
    Returns fraction (negative = loss)."""
    return 2 * np.sqrt(price_ratio) / (1 + price_ratio) - 1

# IL across price changes
ratios = np.linspace(0.1, 5.0, 500)
il_values = [impermanent_loss(r) * 100 for r in ratios]

plt.figure(figsize=(10, 6))
plt.plot(ratios, il_values, 'b-', linewidth=2)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.axvline(x=1.0, color='green', linestyle='--', alpha=0.5, label='No price change')

# Mark key points
key_ratios = [0.5, 0.75, 1.25, 1.5, 2.0, 3.0, 5.0]
for r in key_ratios:
    il = impermanent_loss(r) * 100
    plt.plot(r, il, 'ro', markersize=6)
    plt.annotate(f'{il:.1f}%', (r, il), textcoords='offset points',
                 xytext=(5, 10), fontsize=8)

plt.xlabel('Price Ratio (new / initial)')
plt.ylabel('Impermanent Loss %')
plt.title('Impermanent Loss vs Price Change')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("Impermanent loss at key price changes:")
print(f"{'Price Change':>15s} {'IL':>8s}")
for r in key_ratios:
    il = impermanent_loss(r) * 100
    direction = '↓' if r < 1 else '↑' if r > 1 else '—'
    print(f"  {r:>5.1f}x ({direction:1s})     {il:>6.2f}%")
print(f"\nIL is always negative. LPs must earn enough fees to offset it.")

In [ ]:
# Compute impermanent loss for each pool in the dataset
print("Impermanent Loss by Pool:\n")

for pool_name in amm_pools['pool'].unique():
    pool_data = amm_pools[amm_pools['pool'] == pool_name].sort_values('state_index')
    
    # Initial and final prices
    initial_price = pool_data['price_a_in_b'].iloc[0]
    final_price = pool_data['price_a_in_b'].iloc[-1]
    price_ratio = final_price / initial_price
    il = impermanent_loss(price_ratio) * 100
    
    # Fee revenue estimate (sum of fees)
    total_fees = pool_data['fee'].sum()
    avg_tvl = pool_data['pool_tvl'].mean()
    fee_yield = total_fees / avg_tvl * 100 if avg_tvl > 0 else 0
    
    net = fee_yield + il  # IL is negative, fees are positive
    
    print(f"  {pool_name}")
    print(f"    Price change: {initial_price:.4f} → {final_price:.4f} ({price_ratio:.3f}x)")
    print(f"    Impermanent loss: {il:.2f}%")
    print(f"    Fee yield: +{fee_yield:.2f}%")
    print(f"    Net LP return: {net:+.2f}% {'(profitable)' if net > 0 else '(underwater)'}")
    print()

## LP Token Economics

When you provide liquidity, you receive **LP tokens** — receipts representing your share of the pool. These tokens are themselves ERC-20 fungible tokens. Your share of the pool (and the fees it collects) is proportional to your LP token holdings.

In [ ]:
# Track LP token distribution from the dataset
# Analyze add/remove liquidity events
liq_events = amm_pools[amm_pools['action'].isin(['add_liquidity', 'remove_liquidity'])].copy()

print(f"Liquidity events: {len(liq_events)}")
print(f"  Adds: {(liq_events['action'] == 'add_liquidity').sum()}")
print(f"  Removes: {(liq_events['action'] == 'remove_liquidity').sum()}")

# LP token over time per pool
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
for ax, pool_name in zip(axes.flat, amm_pools['pool'].unique()):
    pool_data = amm_pools[amm_pools['pool'] == pool_name]
    ax.plot(pool_data['state_index'], pool_data['total_lp_tokens'], 'purple', linewidth=1.5)
    
    # Mark add/remove events
    adds = pool_data[pool_data['action'] == 'add_liquidity']
    removes = pool_data[pool_data['action'] == 'remove_liquidity']
    ax.plot(adds['state_index'], adds['total_lp_tokens'], 'g^', markersize=8, label='Add')
    ax.plot(removes['state_index'], removes['total_lp_tokens'], 'rv', markersize=8, label='Remove')
    
    ax.set_title(pool_name, fontsize=10)
    ax.set_xlabel('State index')
    ax.set_ylabel('Total LP Tokens')
    ax.legend(fontsize=8)
    ax.grid(alpha=0.3)

plt.suptitle('LP Token Supply Over Time', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# Demonstrate LP value tracking with our simulated pool
# What would providers get back if they withdrew now?

print("LP Provider Returns (simulated pool):")
print(f"{'Provider':15s} {'LP Tokens':>10s} {'Share':>7s} {'Iron Out':>9s} {'Salt Out':>9s}")

for provider in ['Brenn Auster', 'Torren Gael', 'Dessa Morin']:
    lp = pool.lp_balances[provider]
    share = lp / pool.total_lp_tokens
    iron_out = pool.reserve_a * share
    salt_out = pool.reserve_b * share
    print(f"{provider:15s} {lp:10.2f} {share:6.1%} {iron_out:9.1f} {salt_out:9.1f}")

# Compare to initial deposits
print(f"\nComparison to initial deposits:")
print(f"{'Provider':15s} {'Init Iron':>10s} {'Now Iron':>9s} {'Init Salt':>10s} {'Now Salt':>9s}")
for name, a, b in providers:
    lp = pool.lp_balances[name]
    share = lp / pool.total_lp_tokens
    print(f"{name:15s} {a:10.0f} {pool.reserve_a * share:9.1f} "
          f"{b:10.0f} {pool.reserve_b * share:9.1f}")

print(f"\nProviders now hold a different ratio of assets.")
print(f"If the price moved, they may have less total value than holding — that's impermanent loss.")

## TVL and Pool Health

**Total Value Locked (TVL)** is the primary health metric for AMM pools. Higher TVL means lower slippage and better prices for traders.

In [ ]:
# TVL over time
fig, ax = plt.subplots(figsize=(12, 5))

for pool_name in amm_pools['pool'].unique():
    pool_data = amm_pools[amm_pools['pool'] == pool_name]
    ax.plot(pool_data['state_index'], pool_data['pool_tvl'], linewidth=1.5, label=pool_name)

ax.set_xlabel('State Index')
ax.set_ylabel('Total Value Locked')
ax.set_title('TVL Over Time by Pool')
ax.legend(fontsize=8)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Final TVL ranking
final_tvl = amm_pools.groupby('pool')['pool_tvl'].last().sort_values(ascending=False)
print("Current pool TVL ranking:")
for pool, tvl in final_tvl.items():
    print(f"  {pool:35s} TVL = {tvl:,.0f}")

## The AMM vs. Traditional Markets

How does the AMM compare to the inspector-based system from Tutorial 01?

In [ ]:
# Compare: guild trades with inspectors vs AMM
print("=== TRADITIONAL GUILD MARKET (from guild_trade_ledger.csv) ===")
dispute_rate = (guild_trades['outcome'] != 'completed').mean()
fraud_rate = guild_trades['fraud_detected'].mean()
bribe_rate = (guild_trades['inspector_ruling'] == 'inspector_bribed').sum() / len(guild_trades)
print(f"  Dispute rate: {dispute_rate:.1%}")
print(f"  Fraud rate: {fraud_rate:.1%}")
print(f"  Inspector bribery: {bribe_rate:.1%}")
print(f"  Requires: inspectors, judges, enforcement")

print(f"\n=== AMM-BASED EXCHANGE (from amm_liquidity_pools.csv) ===")
amm_swaps = amm_pools[amm_pools['action'] == 'swap']
print(f"  Avg slippage: {amm_swaps['slippage_pct'].mean():.2f}%")
print(f"  Avg fee: {amm_swaps['fee'].mean():.2f} per swap")
print(f"  Dispute rate: 0% (automated, deterministic)")
print(f"  Fraud rate: 0% (math, not trust)")
print(f"  Requires: liquidity providers, the formula")

print(f"\nThe tradeoff:")
print(f"  Traditional: lower cost per trade, but enforcement failures")
print(f"  AMM: slippage + fees, but zero disputes and zero fraud")
print(f"  Near the Dens, AMM wins. At the Capital, traditional may suffice.")

## Exercises

### Exercise 1: Optimal Pool Depth

How much liquidity is needed to keep slippage below 1% for a 100-unit trade? Vary the pool size from 500 to 10,000 and plot the resulting slippage.

In [ ]:
# Slippage as a function of pool depth
trade_size = 100
pool_sizes = np.linspace(500, 10000, 50)
slippages = []

for size in pool_sizes:
    # Equal reserves: x = y = size/2
    reserve = size / 2
    k = reserve * reserve
    
    # Swap: deposit trade_size of B, get A
    new_b = reserve + trade_size * 0.997  # After fee
    new_a = k / new_b
    amount_out = reserve - new_a
    
    spot_price = 1.0  # Equal reserves → 1:1
    exec_price = amount_out / trade_size
    slip = (spot_price - exec_price) / spot_price * 100
    slippages.append(slip)

plt.figure(figsize=(10, 5))
plt.plot(pool_sizes, slippages, 'b-', linewidth=2)
plt.axhline(y=1.0, color='firebrick', linestyle='--', label='1% slippage target')
plt.xlabel('Pool TVL (total reserves)')
plt.ylabel('Slippage % for 100-unit trade')
plt.title('Pool Depth vs Slippage')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Find crossover
for size, slip in zip(pool_sizes, slippages):
    if slip <= 1.0:
        print(f"Slippage drops below 1% at pool TVL ≈ {size:,.0f}")
        print(f"Rule of thumb: pool TVL should be ~{size/trade_size:.0f}x the typical trade size")
        break

### Exercise 2: Fee Sensitivity

The standard AMM fee is 0.3%. How does changing the fee affect LP profitability vs. trader costs? Test fees from 0.05% to 2% and find the fee that maximizes LP returns while keeping trader costs reasonable.

In [ ]:
# Fee sensitivity simulation
np.random.seed(42)
fee_rates = np.linspace(0.0005, 0.02, 20)
lp_returns = []
trader_costs = []
n_trades = 100

for fee_rate in fee_rates:
    # Simulate pool with this fee rate
    reserve_a = 1000.0
    reserve_b = 1000.0
    total_fees = 0
    total_trader_cost = 0
    
    for _ in range(n_trades):
        amount_in = np.random.uniform(5, 30)
        fee = amount_in * fee_rate
        net_in = amount_in - fee
        total_fees += fee
        
        k = reserve_a * reserve_b
        new_a = reserve_a + net_in
        new_b = k / new_a
        amount_out = reserve_b - new_b
        
        # Trader cost = ideal amount out - actual amount out
        ideal_out = amount_in * (reserve_b / reserve_a)
        total_trader_cost += (ideal_out - amount_out)
        
        reserve_a = new_a + fee  # Fees stay in pool
        reserve_b = new_b
    
    lp_returns.append(total_fees / 2000 * 100)  # As % of initial TVL
    trader_costs.append(total_trader_cost / n_trades)  # Avg per trade

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(fee_rates * 100, lp_returns, 'g-', linewidth=2)
ax1.axvline(x=0.3, color='steelblue', linestyle='--', label='Uniswap default (0.3%)')
ax1.set_xlabel('Fee Rate %')
ax1.set_ylabel('LP Return (% of TVL)')
ax1.set_title('LP Returns vs Fee Rate')
ax1.legend()
ax1.grid(alpha=0.3)

ax2.plot(fee_rates * 100, trader_costs, 'r-', linewidth=2)
ax2.axvline(x=0.3, color='steelblue', linestyle='--', label='Uniswap default (0.3%)')
ax2.set_xlabel('Fee Rate %')
ax2.set_ylabel('Avg Trader Cost per Swap')
ax2.set_title('Trader Cost vs Fee Rate')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"At 0.3% fee: LP return = {lp_returns[np.argmin(np.abs(fee_rates - 0.003))]:.2f}% of TVL")
print(f"Higher fees → more LP revenue, but traders go elsewhere.")
print(f"The market finds equilibrium: fees high enough to attract LPs, low enough to attract traders.")

### Exercise 3: Arbitrage and Price Discovery

If the AMM price diverges from the "true" market price, arbitrageurs can profit by trading until the prices converge. Implement an arbitrage function.

In [ ]:
def compute_arbitrage(amm_price, market_price, reserve_a, reserve_b, fee_rate=0.003):
    """Calculate optimal arbitrage trade when AMM price != market price.
    amm_price = reserve_b / reserve_a (price of A in terms of B)
    market_price = true price of A in terms of B
    Returns (direction, optimal_amount, profit)."""
    
    if abs(amm_price - market_price) / market_price < fee_rate:
        return ('none', 0, 0)  # Not profitable after fees
    
    k = reserve_a * reserve_b
    
    if amm_price < market_price:
        # A is cheap on AMM — buy A with B
        # Target: new_price = market_price → new_b / new_a = market_price
        target_a = (k / market_price) ** 0.5
        amount_b_in = (k / target_a - reserve_b) / (1 - fee_rate)
        if amount_b_in <= 0:
            return ('none', 0, 0)
        amount_a_out = reserve_a - target_a
        profit = amount_a_out * market_price - amount_b_in
        return ('buy_A', amount_b_in, profit)
    else:
        # A is expensive on AMM — sell A for B
        target_a = (k / market_price) ** 0.5
        amount_a_in = (target_a - reserve_a) / (1 - fee_rate)
        if amount_a_in <= 0:
            return ('none', 0, 0)
        amount_b_out = reserve_b - k / target_a
        profit = amount_b_out - amount_a_in * market_price
        return ('sell_A', amount_a_in, profit)

# Demo: AMM has iron_ore at 1.6 salt, but market says 2.0
amm_price = 1.6
market_price = 2.0
reserve_a, reserve_b = 1000, 1600  # Consistent with amm_price = 1.6

direction, amount, profit = compute_arbitrage(amm_price, market_price, reserve_a, reserve_b)

print(f"Arbitrage opportunity:")
print(f"  AMM price: {amm_price} salt per iron")
print(f"  Market price: {market_price} salt per iron")
print(f"  Direction: {direction}")
print(f"  Optimal trade: {amount:.1f}")
print(f"  Expected profit: {profit:.1f} salt_crystal")
print(f"\nArbitrage is the mechanism that keeps AMM prices aligned with reality.")
print(f"Without arbitrageurs, AMM prices would drift from market prices.")

## Summary

In this tutorial, we learned:

1. **The constant product formula** (x × y = k) creates an automated market with continuous liquidity
2. **Swaps** change the reserve ratio — buying A means depositing B — and the price adjusts automatically
3. **Slippage** scales with trade size relative to pool depth — large trades get worse prices
4. **Impermanent loss** is the cost LPs pay when prices move — they end up worse than holding
5. **LP tokens** represent pool shares and give providers claim on reserves plus accumulated fees

**Key insight:** The AMM eliminates the need for trusted market-makers, inspectors, or price oracles. It replaces institutional trust with mathematical constraint. Near the Dens, where every human intermediary can be corrupted, the formula holds. The price is the ratio. The curve is the market.

But the AMM introduces a new vulnerability: **Miner Extractable Value (MEV)**. Anyone who can see pending transactions can front-run them — buying just before a large order to profit from the price impact. This is the topic of Tutorial 07.

---

**Next Tutorial:** MEV and Front-Running — when validators extract value from transaction ordering.

---

> *"The exchange is trustless. But the queue to reach it is not. He who orders the line extracts from those who stand in it."*
>
> — Brenn Auster